In [1]:
from shared.utils import load_mnist
import numpy as np

In [2]:
X_train, y_train = load_mnist('./mnist/', kind='train')

In [3]:
X_test, y_test = load_mnist('./mnist/', kind='t10k')

In [4]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)
X_train_centered = (X_train - mean_vals) / std_val
X_test_centered = (X_test - mean_vals) / std_val
del X_train, X_test

In [5]:
print(X_train_centered.shape, y_train.shape)
print(X_test_centered.shape, y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [7]:
import tensorflow as tf
import tensorflow.keras as keras

In [10]:
random_seed = 123
np.random.seed(random_seed)
tf.set_random_seed(random_seed)

In [11]:
y_train_onehot = keras.utils.to_categorical(y_train)
print('First 3 labels: ', y_train[:3])
print('First 3 labels (one-hot):\n', y_train_onehot[:3])

First 3 labels:  [5 0 4]
First 3 labels (one-hot):
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(units=50, input_dim=X_train_centered.shape[1], kernel_initializer='glorot_uniform', bias_initializer='zeros', activation='tanh'))
model.add(keras.layers.Dense(units=50, input_dim=50, kernel_initializer='glorot_uniform', bias_initializer='zeros', activation='tanh'))
model.add(keras.layers.Dense(units=y_train_onehot.shape[1], input_dim=50, kernel_initializer='glorot_uniform', bias_initializer='zeros', activation='softmax'))
sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=.9)
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
history = model.fit(X_train_centered, y_train_onehot, batch_size=64, epochs=50, verbose=1, validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
54000/54000 [==============================] - 3s 63us/sample - loss: 0.7422 - val_loss: 0.3736
Epoch 2/50
54000/54000 [==============================] - 3s 58us/sample - loss: 0.3781 - val_loss: 0.2805
Epoch 3/50
54000/54000 [==============================] - 3s 52us/sample - loss: 0.3109 - val_loss: 0.2411
Epoch 4/50
54000/54000 [==============================] - ETA: 0s - loss: 0.273 - 3s 50us/sample - loss: 0.2737 - val_loss: 0.2171
Epoch 5/50
54000/54000 [==============================] - 4s 67us/sample - loss: 0.2478 - val_loss: 0.1999
Epoch 6/50
54000/54000 [==============================] - 3s 46us/sample - loss: 0.2276 - val_loss: 0.1842
Epoch 7/50
54000/54000 [==============================] - 3s 59us/sample - loss: 0.2110 - val_loss: 0.1732
Epoch 8/50
54000/54000 [==============================] - 3s 56us/sample - loss: 0.1973 - val_loss: 0.1640
Epoch 9/50
54000/54000 

In [15]:
y_train_pred = model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(y_train == y_train_pred, axis=0)
train_acc = correct_preds / y_train.shape[0]
print('First 3 predictions: ', y_train_pred[:3])
print('Training accuracy: %.2f%%' % (train_acc * 100))

First 3 predictions:  [5 0 4]
Training accuracy: 98.81%


In [16]:
y_test_pred = model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(y_test == y_test_pred, axis=0)
test_acc = correct_preds / y_test.shape[0]
print('Test accuracy: %.2f%%' % (test_acc * 100))

Test accuracy: 96.27%
